In [1]:
import json
import subprocess
import shutil
import os, glob
import time
from tqdm import tqdm
from shapely.geometry import Polygon, box

In [2]:
state = 'arkansas'

In [32]:
# name of the state we are working with
working_directory_path = "G:\\My Drive\\states\\{}".format(state)
img_path               = "{}\\images".format(working_directory_path)

processing             = "{}\\processing".format(img_path)

# name of the filtered file based on state name example: alaska_filtered.geojson
filtered_files_path    = "G:\\My Drive\\states\\{}\\counties_filtered".format(state)

# Note: This paths might need to be change depending on where you installed the software
# paths for allmapsoft program
downloader_path        = "C:\\allmapsoft\\gsmd\\downloader.exe "
combiner_path          = "C:\\allmapsoft\\gsmd\\combiner.exe "

# Create images directory if it doesn't exist
os.makedirs(img_path, exist_ok=True)
os.makedirs(processing, exist_ok=True)


print("Paths:")
print("--working_directory_path : " + working_directory_path)
print("--img_path               : " + img_path)
print("--processing             : " + processing)
print("--filtered_files_path    : " + filtered_files_path)

print("--downloader_path        : " + downloader_path)
print("--combiner_path          : " + combiner_path)

Paths:
--working_directory_path : G:\My Drive\states\arkansas
--img_path               : G:\My Drive\states\arkansas\images
--processing             : G:\My Drive\states\arkansas\images\processing
--filtered_files_path    : G:\My Drive\states\arkansas\counties_filtered
--downloader_path        : C:\allmapsoft\gsmd\downloader.exe 
--combiner_path          : C:\allmapsoft\gsmd\combiner.exe 


In [28]:
def get_square_bounds(polygon):
    # Convert the input polygon to a shapely Polygon
    shapely_polygon = Polygon(polygon[0])
    
    # Get the bounding box of the polygon
    bbox = shapely_polygon.bounds
    
    # Extract the left, right, top and bottom coordinates
    left, bottom, right, top = bbox
    return f"{left} {right} {top} {bottom}"

In [29]:
user = "TAs_office-Porfirios_desktop"

In [ ]:
# Get a list of all the GeoJSON files in the directory
geojson_files = glob.glob(os.path.join(filtered_files_path, "*.geojson"))
#print(len(geojson_files))
# Iterate through each file
for geojson_file in geojson_files:
    
    print(geojson_file)
    county_name = os.path.basename(geojson_file).split('_')[2].split('.')[0] + "_County"
    print(f"Processing: {county_name}")
    
    
    processing_filepath = processing + "\\{}".format(county_name)+".txt"
    # check if the file is being processed  
    if os.path.exists(processing_filepath):
        # check if you created the processing file
        # we do this in case there was an error 
        # and you did not finihsed processing the file
        # skip the file if you did not make it
        with open(processing_filepath, 'r') as p:
            p_user = p.read()
        p.close()
        if p_user != user:
            print("--"+county_name +" is being processed by: " + p_user)
            continue
    else:
        #create a temp file 
        with open(processing_filepath, 'w') as t:
            t.write(user)

    
    # Open the GeoJSON file
    with open(geojson_file, 'r') as f:
        data = json.load(f)
    
    temp = "{}\\{}\\temp".format(img_path,county_name)
    os.makedirs(temp, exist_ok=True)
    
    dst  = os.path.join(img_path, county_name)
    # Create images directory if it doesn't exist
    os.makedirs(dst, exist_ok=True)
    
    # Get the checkpoint file for this GeoJSON file
    checkpoint_file = dst +"\\checkpoint.txt"
    if os.path.exists(checkpoint_file):
        with open(checkpoint_file, 'r') as f:
            start_index = int(f.readline().strip())
            print("Processed images: " + str(start_index))
    else:
        start_index = 0

    # Iterate through the features in the GeoJSON file
    for num, feature in enumerate(tqdm(data['features'][start_index:], desc='Processing data')):
        # Get the coordinates of the current feature
        coordinates = feature['geometry']['coordinates']
        i = num + start_index

        # Format = [left longitude, right longitude, top latitude, bottom latitude]
        square = get_square_bounds(coordinates)

        filename = f"{county_name}_task{i}.jpg"

        # Generate command
        gmid_name = county_name + "_" + f"task{i}.gmid"
        para2 = "1"
        para3 = "21"
        command = downloader_path + '"'+gmid_name +'\" ' + para2 +" "+ para3 + " "+ square +' "'+ temp +'"'
        # Call cmd and run downloader program
        subprocess.run(command, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        combine = combiner_path +'"' + temp +"\\"+ gmid_name +"\" jpg meters"

        # Call cmd and run combine program
        subprocess.run(combine, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

        # Set the source and destination directories
        name = county_name + f"_task{i}"
        img  = name + ".jpg"
        src  = os.path.join(temp, name+"_combined")

        # Set the source and destination file paths
        src_file = os.path.join(src, img)
        dst_file = os.path.join(dst, filename)

        # Use the shutil module to move the file
        shutil.move(src_file, dst_file)

        # Clean
        # -----Generate command to delete path
        folder_path = os.path.join(img_path, name)
        combined_folder_path = os.path.join(img_path, name+"_combined")

        # -----Use the remove function to delete the path
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                path = os.path.join(root,file)
                os.chmod(path,0o777)
                
        #shutil.rmtree(folder_path)
        #shutil.rmtree(combined_folder_path)

        # Use glob to get a list of all the files in the directory
        del_dir = os.path.join(working_directory_path, img_path)
        files   = glob.glob(os.path.join(del_dir, '*'))

        # Iterate over the list of files and delete them one by one
        for file in files:
            if os.path.isfile(file):
                # If it's a regular file, delete it
                os.remove(file)
            # Save the checkpoint
        with open(checkpoint_file, 'w') as f:
            f.write(str(num + start_index))
    os.remove(processing_filepath)

G:\My Drive\states\arkansas\counties_filtered\filtered_Arkansas_Arkansas.geojson
Processing: Arkansas_County
Processed images: 2689


Processing data:  14%|█▍        | 1286/8871 [5:37:07<36:01:34, 17.10s/it]